In [44]:
# instruciton datasetを今回のSFT形式に変換する

In [1]:
import os

#dataフォルダ内をリセット
os.system("rm -rf data/*")

0

In [2]:
from datasets import load_dataset
import json

In [3]:
"""
本番はこっち｡
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい
"""

#question_template="### 指示：以下の質問に答えなさい。 ### 質問："
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n### 応答:\n"

# dolly

In [4]:
d_dataset=load_dataset("kunishou/databricks-dolly-15k-ja")["train"]

In [5]:
len(d_dataset)

15015

In [12]:
#question_template2="### 指示：以下の質問に答えなさい。"
#question_template2="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい｡"
#question_template2="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 質問:\n"
dolly_list=[]
for n_instructions in [1000]:
    output_path=f"data/dolly{n_instructions}.jsonl"
    with open(output_path,"w") as f:
        f.write("")
    loader=iter(d_dataset)
    for i in range(n_instructions+1000):
        original_record=next(loader)
        if "input" in original_record:
            inp=original_record["input"]
        else:
            inp=""
        q=original_record["instruction"]
        a=original_record["output"]
        if len(a)<10:
            print(q)
            print(a)
        if inp=="":
            text=f"{question_template}{q}{answer_template}{a}"
        else:
            text=f"{question_template}{q}\n\n### 入力:\n{inp}\n{answer_template}{a}"
        eval_prefix=""

        line=json.dumps({"text":text},ensure_ascii=False)
        if i>=n_instructions:
            eval_prefix=".eval"
        else:
            dolly_list.append(line)
        with open(output_path+eval_prefix,"a") as f:
            f.write(line+"\n")

dolly_list=list(set(dolly_list))

魚の種類はどっち？イコクエイラクブカとロープ
イコクエイラクブカ
アリスの両親には3人の娘がいる：エイミー、ジェシー、そして三女の名前は？
三女の名前はアリス
「真珠の耳飾りの少女」を描いたオランダの画家は？
フェルメール
『左きゝの拳銃』でビリー・ザ・キッドを演じたのは？
ポール・ニューマン
海の怪物からアンドロメダを救ったのは誰か
ペルセウス
『氷と炎の歌』シリーズで、カースターク家の創始者は誰でしょう？
カルロン・スターク
1943年のアカデミー賞で最優秀作品賞を受賞した作品は？
カサブランカ
ゴルフでは、アメリカ人はアホウドリを何と呼ぶか
ダブルイーグル
魚の一種はどちらでしょう？シードラゴン、レッドベアード
シードラゴン
ルイス・ハミルトンは何回F1チャンピオンになったか？
7回
魚の一種はどれ？モーラモーラまたはモーラ
モーラモーラ
猫には9つの命がある？
いいえ
ケララ州で炭疽菌の症例が見つかったのはどの動物か？
野生のイノシシ
提供された文章から、テイラー・スウィフトがリリースした2枚目のスタジオアルバムを抽出せよ。
フィアレス
1991年シーズン終了時にリーグから降格したチームはどこでしょう？
ジスボーン市
鳥と魚はどっちだ：黒珍、黒尉鶲
黒珍は魚、黒鶲は鳥
コッカースパニエルとプードルのミックス犬種は？
コッカプー
靴ひもの先端にある小さな部品は何と呼ばれているのでしょうか？
アグレットです。
ブラジル、ウルグアイ、モザンビーク、アンゴラの4カ国を結ぶもの
ポルトガルの植民地
人類初のテクノロジーとされるのはどれ？
火
Sで始まる単語は次のうちどれでしょう：Snake, Towel, Table, Shower, Yam, Butterfly, Apple
スネーク、シャワー
ウォルター・ケーニグが『スタートレック』シリーズで演じた役柄は？
チェコフ少尉
ワシントン州最大の都市は？
シアトル
ビンゴゲームで、「2羽の小鴨」というフレーズで表される数字はどれでしょう？
22
フレディ・マーキュリーの本名は？
ファロク・ブルサラ
シェイクスピアの戯曲の中で一番長いのはどれ？
ハムレットです。
魚の種類はどちらでしょう？ナースまたはナースシャーク
ナースシャーク
イタリアンファーストが発売されたテキスト年号より抜粋
2013
マイケル・ジョーダンがプレ

In [8]:
a

'キャッチング・ファイア』『モッキングジェイ』『越境者グレゴール』『イヤー・オブ・ジャングル』『ソングバード・アンド・スネークスのバラード'

# Jaster
- コンペでは使ってはいけないので注意

In [13]:
import random
jaster_data_list=[]
path="../4_eval/llm-leaderboard/artifacts/jaster:v3/jaster/1.1.0/evaluation/train/jcommonsenseqa.json"
with open(path,"r") as f:
    records=json.load(f)
instruction=records["instruction"]  
for record in records["samples"]:
    d={
        "text":instruction+"\n"+record["input"],
        "output":record["output"]
    }
    jaster_data_list.append(d)

random.seed(0)
random.shuffle(jaster_data_list)
len(jaster_data_list)

8045

In [17]:
import pandas as pd
#df=pd.DataFrame(jaster_data_list)
#df.to_csv("data/jaster.csv")

In [40]:
for n_instructions in [100,1000,2500,5000,10000,50000,140000]:
    output_path=f"data/jaster_{n_instructions}.jsonl"
    with open(output_path,"w") as f:
        f.write("")
    loader=iter(jaster_data_list)
    jaster_list=[]
    for i in range(n_instructions+300):
        original_record=next(loader)
        q=original_record["text"]
        a=original_record["output"]
        text=f"{question_template}{q}{answer_template}{a}"

        line=json.dumps({"text":text},ensure_ascii=False)

        eval_prefix=""
        if i>=n_instructions:
            eval_prefix=".eval"
        else:
            jaster_list.append(line)
 
        with open(output_path+eval_prefix,"a") as f:
            f.write(line+"\n")